In [1]:
!pip install deep_sort_realtime pycocotools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 74.2 MB/s eta 0:00:00:00:0100:01


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import scipy.io
import shutil

In [3]:
# Enable Mixed Precision Training
set_global_policy('mixed_float16')

# Load Retail Product Checkout (RPC) Dataset
DATASET_PATH = "/kaggle/input/retail-product-checkout-dataset"
TRAIN_DIR = os.path.join(DATASET_PATH, "train2019")
VAL_DIR = os.path.join(DATASET_PATH, "val2019")
TEST_DIR = os.path.join(DATASET_PATH, "test2019")
ANNOTATIONS_TRAIN = os.path.join(DATASET_PATH, "instances_train2019.json")
ANNOTATIONS_VAL = os.path.join(DATASET_PATH, "instances_val2019.json")
ANNOTATIONS_TEST = os.path.join(DATASET_PATH, "instances_test2019.json")

In [4]:
# Load COCO Annotations
def load_coco_annotations(annotation_file):
    coco = COCO(annotation_file)
    return coco

coco_train = load_coco_annotations(ANNOTATIONS_TRAIN)
coco_val = load_coco_annotations(ANNOTATIONS_VAL)

loading annotations into memory...
Done (t=0.82s)
creating index...
index created!
loading annotations into memory...
Done (t=0.93s)
creating index...
index created!


In [5]:
# Custom Data Generator for COCO Dataset
class CocoDataGenerator(Sequence):
    def __init__(self, coco, img_dir, batch_size=32, img_size=(224, 224), shuffle=True):
        self.coco = coco
        self.img_dir = img_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.image_ids = list(self.coco.imgs.keys())
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_ids) / self.batch_size))

    def __getitem__(self, index):
        start_idx = index * self.batch_size
        end_idx = min((index + 1) * self.batch_size, len(self.image_ids))
        indexes = self.indexes[start_idx:end_idx]
        
        if len(indexes) == 0:
            return np.empty((0, *self.img_size, 3)), np.empty((0, NUM_CLASSES))
        
        image_ids_temp = [self.image_ids[k] for k in indexes]
        X, y = self.__data_generation(image_ids_temp)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_ids))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, image_ids_temp):
        X = np.empty((self.batch_size, *self.img_size, 3))
        y = np.empty((self.batch_size, 1), dtype=int)

        for i, img_id in enumerate(image_ids_temp):
            img_info = self.coco.imgs[img_id]
            img_path = os.path.join(self.img_dir, img_info['file_name'])
            img = cv2.imread(img_path)
            img = cv2.resize(img, self.img_size)
            img = img / 255.0
            X[i,] = img

            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            anns = self.coco.loadAnns(ann_ids)
            category_id = anns[0]['category_id'] if anns else 0

            # Ensure category_id is within bounds
            category_id = min(category_id, len(self.coco.cats) - 1)
            y[i] = category_id

        return X, tf.keras.utils.to_categorical(y, num_classes=len(self.coco.cats))


In [6]:
# Image Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
NUM_CLASSES = len(coco_train.cats)

# Create Data Generators
train_generator = CocoDataGenerator(coco_train, TRAIN_DIR, batch_size=BATCH_SIZE, img_size=IMG_SIZE)
val_generator = CocoDataGenerator(coco_val, VAL_DIR, batch_size=BATCH_SIZE, img_size=IMG_SIZE)
train_generator.image_ids = train_generator.image_ids[:1000]  # Use only 5000 images
val_generator.image_ids = val_generator.image_ids[:100]

In [7]:
print(f"Total train images: {len(train_generator.image_ids)}")
print(f"Total val images: {len(val_generator.image_ids)}")

Total train images: 1000
Total val images: 100


In [8]:
# Image Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
NUM_CLASSES = len(coco_train.cats)

# Create Data Generators
train_generator = CocoDataGenerator(coco_train, TRAIN_DIR, batch_size=BATCH_SIZE, img_size=IMG_SIZE)
val_generator = CocoDataGenerator(coco_val, VAL_DIR, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

# Load Pretrained MobileNetV2 (Optimized with alpha=0.35 for speed)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3), alpha=0.35)
base_model.trainable = False  # Freeze base model

# Add Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
out = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model with reduced epochs
EPOCHS = 1
history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)

2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3358/3358 ━━━━━━━━━━━━━━━━━━━━ 1663s 491ms/step - accuracy: 0.4845 - loss: 2.1494 - val_accuracy: 0.0805 - val_loss: 10.6958


In [9]:
# Save Model
model.save("mobilenetv2_rpc_optimized.h5")

In [10]:
# Inference on Sample Image
def predict_image(image_path, model):
    img = cv2.imread(image_path)
    img = cv2.resize(img, IMG_SIZE)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions)
    
    plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
    plt.title(f"Predicted Class: {predicted_class}")
    plt.show()

In [11]:
# Initialize DeepSORT Tracker
tracker = DeepSort(max_age=30, n_init=3)

/usr/local/lib/python3.10/dist-packages/deep_sort_realtime/embedder/embedder_pytorch.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.lo

In [12]:
import cv2
from google.colab.patches import cv2_imshow

In [13]:
# Object Tracking Function
def track_objects(video_path, model):
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        img_resized = cv2.resize(frame, IMG_SIZE) / 255.0
        img_resized = np.expand_dims(img_resized, axis=0)
        
        predictions = model.predict(img_resized)
        predicted_class = np.argmax(predictions)
        
        detections = [(50, 50, 200, 200, 0.9, predicted_class)]  # Mock detection
        tracks = tracker.update_tracks(detections, frame=frame)
        
        for track in tracks:
            if not track.is_confirmed():
                continue
            bbox = track.to_tlbr()
            track_id = track.track_id
            
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
            cv2.putText(frame, f'ID {track_id}', (int(bbox[0]), int(bbox[1]) - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        cv2_imshow(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [14]:
# Run Tracking on Sample Video
SAMPLE_VIDEO_PATH = "/kaggle/input/samplevid"
track_objects(SAMPLE_VIDEO_PATH, model)
